In [1]:
import transformers

c:\Users\17539\anaconda3\envs\nlpasm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
model_name = "D:/models/Qwen2.5-7B-Instruct"

# 创建模型实例
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# 取出对应的模型词嵌入
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  8.82it/s]


In [11]:
a = '我是一个大学生，正在学习如何微调大模型，你能简单介绍一下相关的方法嘛？'
t = tokenizer(a, return_tensors="pt", padding="max_length", truncation=True, max_length=512)
t.attention_mask, t.input_ids

(tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 

In [14]:
t_tensor = t.input_ids
t_masking = t.attention_mask
t_tensor[t_masking == 0] = -100
t_tensor

tensor([[ 35946, 101909, 104422,   3837,  96555, 100134, 100007,  48934,  47872,
          26288, 104949,   3837, 107809, 100405, 109432,  78556, 104339, 102455,
          11319,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -

In [4]:
prompt = "给我一个大模型的简单介绍"
messages = [
    {"role": "system", "content": "你是通译千问，由阿里云团队研发。 你现在是一个聊天助手"},
    {"role": "user", "content": prompt},
]

# 对prompt的内容进行切分
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)

# 转换为pytorch的tensor
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# 将输入进模型进行推理
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
)

# 将input与generated_output进行拼接，然后取出输出，从input长度的后面开始取起
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

# 再使用tokenizer进行decode
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

response

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


'当然可以！大模型（也称为大规模预训练模型）是指参数量非常大的机器学习模型，通常包含数亿甚至数十亿个参数。这些模型通过在大量未标注的数据上进行预训练，然后通过少量标注数据进行微调，来完成各种自然语言处理任务。\n\n大模型的主要特点包括：\n\n1. **大规模参数**：参数量巨大，这使得模型能够捕捉到更复杂的特征和模式。\n2. **预训练-微调机制**：首先在大量的文本数据上进行预训练，然后根据具体任务进行微调。\n3. **泛化能力**：由于训练数据量大且多样，大模型通常具有较强的泛化能力，可以在多种下游任务中表现出色。\n4. **多任务处理**：经过适当微调后，大模型可以用于多种不同的自然语言处理任务，如文本分类、情感分析、机器翻译等。\n\n目前，一些知名的大型语言模型包括谷歌的BERT、Transformer系列模型、阿里巴巴的M6等。这些模型在学术界和工业界都有广泛的应用，并推动了自然语言处理领域的发展。'